In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For VGGNet, the lr starts from 0.01, and is divided by 10 at 50 and 100 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])

# This cell won't be given, but students will complete the training

lr = 0.0001
weight_decay = 1e-4
epochs = 30
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.761 (0.761)	Data 0.279 (0.279)	Loss 0.3891 (0.3891)	Prec 85.156% (85.156%)
Epoch: [0][100/391]	Time 0.038 (0.050)	Data 0.002 (0.005)	Loss 0.3677 (0.4361)	Prec 89.844% (85.257%)
Epoch: [0][200/391]	Time 0.042 (0.046)	Data 0.002 (0.004)	Loss 0.3244 (0.4372)	Prec 90.625% (85.316%)
Epoch: [0][300/391]	Time 0.050 (0.045)	Data 0.003 (0.003)	Loss 0.4105 (0.4343)	Prec 85.938% (85.312%)
Validation starts
Test: [0/79]	Time 0.259 (0.259)	Loss 0.4329 (0.4329)	Prec 84.375% (84.375%)
 * Prec 83.620% 
best acc: 83.620000
Epoch: [1][0/391]	Time 0.508 (0.508)	Data 0.449 (0.449)	Loss 0.4842 (0.4842)	Prec 83.594% (83.594%)
Epoch: [1][100/391]	Time 0.037 (0.044)	Data 0.001 (0.007)	Loss 0.3951 (0.4220)	Prec 87.500% (85.605%)
Epoch: [1][200/391]	Time 0.036 (0.040)	Data 0.001 (0.004)	Loss 0.5220 (0.4247)	Prec 82.812% (85.424%)
Epoch: [1][300/391]	Time 0.041 (0.040)	Data 0.002 (0.003)	Loss 0.3980 (0.4265)	Prec 82.031% (85.343%)
Validation starts
Test: [0/79]	Time 0.254 (0.254)	Loss 0.

In [3]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8501/10000 (85%)



In [4]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in[0]) 
    def clear(self):
        self.outputs = []

save_output = SaveOutput()
target_layer = model.features[27]
target_layer.register_forward_pre_hook(save_output)

# Get a sample batch
dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.cuda()

# Forward pass to capture the input
model.eval()
with torch.no_grad():
    _ = model(images)

x = save_output.outputs[0]
print(f"Captured input shape: {x.shape}")

Captured input shape: torch.Size([128, 512, 4, 4])


In [5]:
w_bit = 4
target_conv = model.features[27]
weight_q = target_conv.weight_q.data
w_alpha = target_conv.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha / (2 ** w_bit - 1) 
# divide by alpha first to get normalized values and multiply by (2^w_bit - 1) to get integers
weight_int = torch.round(weight_q / w_alpha * (2 ** w_bit - 1)).int()
print(f"Weight alpha: {w_alpha:.4f}, Weight delta: {w_delta:.4f}")
print(f"Weight int shape: {weight_int.shape}")
print(f"Weight int sample (first few values): {weight_int.flatten()[:10]}")

Weight alpha: 2.8229, Weight delta: 0.1882
Weight int shape: torch.Size([8, 512, 3, 3])
Weight int sample (first few values): tensor([ 0,  2,  0,  9, 11,  9,  6,  6,  6, -2], device='cuda:0',
       dtype=torch.int32)


In [6]:
x_bit = 4    
# Get activation alpha from the target conv layer
target_conv = model.features[27]
x_alpha = target_conv.act_alpha.data.item()
x_delta = x_alpha / (2 ** x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

# x_q divide by alpha first then multiply by (2^x_bit - 1) to get integers
x_int = torch.round(x_q / x_alpha * (2 ** x_bit - 1)).int()
print(f"Activation alpha: {x_alpha:.4f}, Activation delta: {x_delta:.4f}")
print(f"x_int shape: {x_int.shape}")
print(f"x_int sample (first few values): {x_int.flatten()[:10]}")

Activation alpha: 5.3443, Activation delta: 0.3563
x_int shape: torch.Size([128, 512, 4, 4])
x_int sample (first few values): tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0', dtype=torch.int32)


In [7]:
conv_int = torch.nn.Conv2d(
    in_channels=8, 
    out_channels=8,
    kernel_size=3,
    padding=1,
    bias=False
)
conv_int.weight = torch.nn.parameter.Parameter(weight_int.float())

output_int = conv_int(x_int.float()) 

#output_recovered = output_int * x_delta * w_delta
output_recovered = output_int * x_delta * w_delta
print(f"Output recovered shape: {output_recovered.shape}")
print(f"Output recovered sample: {output_recovered[0, 0, :5, :5]}")

Output recovered shape: torch.Size([128, 8, 4, 4])
Output recovered sample: tensor([[ 185.6670,   25.7480, -144.2959, -190.4948],
        [ 135.7803, -138.0601, -367.0427, -408.1456],
        [ 185.1977,  -57.6647, -334.0531, -436.1063],
        [ -73.8913, -324.0624, -489.8150, -440.6659]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [8]:
#### input floating number / weight quantized version
target_conv = model.features[27]   # QuantConv2d(512 → 8)
conv_ref = torch.nn.Conv2d(
    in_channels=512,
    out_channels=8,
    kernel_size=3,
    padding=1,
    bias=False
)
conv_ref.weight = torch.nn.parameter.Parameter(target_conv.weight_q.data) 

output_ref = conv_ref(x)
print(f"Output ref shape: {output_ref.shape}")
print(f"Output ref sample: {output_ref[0, 0, :5, :5]}")    

Output ref shape: torch.Size([128, 8, 4, 4])
Output ref sample: tensor([[ 189.2889,   29.6515, -144.1570, -193.5141],
        [ 133.2846, -139.9815, -372.9612, -410.6110],
        [ 190.7070,  -56.3463, -326.3329, -430.8946],
        [ -72.5739, -328.0229, -488.1766, -438.8649]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [9]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(7.3490, device='cuda:0', grad_fn=<MeanBackward0>)
